In [1]:
#导入需要的包
import numpy as np
import paddle as paddle
import paddle.fluid as fluid
from PIL import Image
import matplotlib.pyplot as plt
import os

In [2]:
#导入数据集
train_reader = paddle.batch(paddle.reader.shuffle(paddle.dataset.mnist.train(),buf_size=512),batch_size=128)
test_reader = paddle.batch(paddle.dataset.mnist.test(),batch_size=128)

## step2:配置网络

In [3]:
#定义多层感知机
def multilayer_perceptron(input):
    hidden1 = fluid.layers.fc(input=input, size=100, act='relu')
    hidden2 = fluid.layers.fc(input=hidden1, size=100,act='relu')
    prediction = fluid.layers.fc(input=hidden2, size=10,act='softmax')
    return prediction

In [4]:
#定义输入输出层
image = fluid.layers.data(name='image',shape=[1,28,28],dtype='float32') #灰度图,单通道,28*28 像素值
label = fluid.layers.data(name='label',shape=[1],dtype='int64') #输出的是图像标签,从 0~9

In [5]:
#获取分类器
model = multilayer_perceptron(image) #感知机实例化并传入参数

In [6]:
#定义 Loss函数,分类问题常用 cross_entropy 函数
cost = fluid.layers.cross_entropy(input=model,label=label) #对 model 应用函数,在标签预测
avg_cost = fluid.layers.mean(cost) #对损失函数求均值
acc = fluid.layers.accuracy(input=model,label=label) #准确率函数,评价输出准确率

In [7]:
#定义优化器
optimizer = fluid.optimizer.AdamOptimizer(learning_rate=0.001) #选用 Adam 算法,学习率指定为 0.001
opts = optimizer.minimize(avg_cost) #使平均损失最小

## step3&step4:模型训练和评估

In [8]:
#定义一个使用 CPU 的解析器
place = fluid.CPUPlace()
exe = fluid.Executor(place)

In [9]:
#参数初始化
exe.run(fluid.default_startup_program())

[]

In [10]:
#定义输入数据维度
feeder = fluid.DataFeeder(place=place,feed_list=[image,label])

In [11]:
#开始训练和测试
for pass_id in range(5):
    #进行训练
    for batch_id, data in enumerate(train_reader()): #遍历 train_reader
        train_cost, train_acc = exe.run(program=fluid.default_main_program(),#运行主程序
                                        feed=feeder.feed(data),#传入数据
                                        fetch_list=[avg_cost,acc]) #误差,准确率
        #每 100 个 batch 打印一次
        if batch_id % 100 == 0:
            print('Pass:%d, Batch:%d, Cost:%.5f,Accuracy:%.5f'%(pass_id,batch_id,train_cost[0],train_acc[0]))
    #进行测试
    test_accs = []
    test_costs = []
    #每训练一轮,进行一次测试
    for batch_id, data in enumerate(test_reader()):#遍历 test_reader
        test_cost,test_acc = exe.run(program=fluid.default_main_program(),#执行训练程序
                                    feed=feeder.feed(data),
                                    fetch_list=[avg_cost,acc])
        test_accs.append(test_acc[0])
        test_costs.append(test_cost[0])
    #求测试结果的平均值
    test_cost = (sum(test_costs)/len(test_costs))
    test_acc = (sum(test_accs)/len(test_accs))
    print('Test:%d,Cost:%.5f,Accuracy:%.5f'%(pass_id,test_cost,test_acc))
    
    #保存 model
    model_save_dir = '/Users/meininghang/hand.inference.model'
    if not os.path.exists(model_save_dir):
        os.makedirs(model_save_dir)
    print('save models to %s'%(model_save_dir))
    fluid.io.save_inference_model(model_save_dir,['image'],[model],exe)

Pass:0, Batch:0, Cost:2.51942,Accuracy:0.12500
Pass:0, Batch:100, Cost:0.36024,Accuracy:0.90625
Pass:0, Batch:200, Cost:0.28533,Accuracy:0.91406
Pass:0, Batch:300, Cost:0.20071,Accuracy:0.93750
Pass:0, Batch:400, Cost:0.23980,Accuracy:0.91406
Test:0,Cost:0.22550,Accuracy:0.93008
save models to /Users/meininghang/hand.inference.model
Pass:1, Batch:0, Cost:0.22111,Accuracy:0.94531
Pass:1, Batch:100, Cost:0.23153,Accuracy:0.92969
Pass:1, Batch:200, Cost:0.25044,Accuracy:0.91406
Pass:1, Batch:300, Cost:0.07422,Accuracy:0.98438
Pass:1, Batch:400, Cost:0.13183,Accuracy:0.96875
Test:1,Cost:0.14740,Accuracy:0.95402
save models to /Users/meininghang/hand.inference.model
Pass:2, Batch:0, Cost:0.19064,Accuracy:0.96094
Pass:2, Batch:100, Cost:0.06469,Accuracy:0.97656
Pass:2, Batch:200, Cost:0.11751,Accuracy:0.96094
Pass:2, Batch:300, Cost:0.09318,Accuracy:0.96875
Pass:2, Batch:400, Cost:0.12731,Accuracy:0.96094
Test:2,Cost:0.12114,Accuracy:0.96143
save models to /Users/meininghang/hand.inference.m

## step5:模型预测

In [13]:
#图像预处理
def load_image(file):
    im = Image.open(file).convert('L') #将 RGB 转化为灰度图像,L 代表灰度图像,灰度图像的像素值在 0~255 之间
    im = im.resize((28,28),Image.ANTIALIAS) #将图像大小更改为 28*28
    im = np.array(im).reshape(1,1,28,28).astype(np.float32) #一维向量化
    im = im / 255.0 * 2.0 - 1.0 #归一化[-1,1]
    return im

In [14]:
infer_exe = fluid.Executor(place)
inference_scope = fluid.core.Scope()

In [ ]:
#加载数据并进行预测
with fluid.scope_guard(inference_scope):
    #获取预先训练好的模型
    #从指定目录中加载 推理 model(inference model)
    [inference_program, feed_target_names, fetch_targets] = fluid.io.load_inference_model(model_save_dir, infer_exe)
    
    img = load

In [15]:
img = Image.open('data/6.png')

FileNotFoundError: [Errno 2] No such file or directory: 'data/6.png'

In [ ]:
#差不多是这个意思,当然还有点问题,比如下载的 mnist 在哪?不过这个不重要,重要的是gan'jue